In [2]:
using OffsetArrays

In [3]:
Nx=5
Ny=5

5

In [4]:
function square_lattice(Nx,Ny)    
    site_idx = range(1,Nx*Ny) 
    lattice = OffsetArray(reshape(site_idx, (Nx,Ny)), 0:Nx-1, 0:Ny-1) |> transpose
    coordinates = []
    for y in 0:Ny-1
        for x in 0:Nx-1
            coordinates = [coordinates; y; x]
        end
    end
    coordinates = reshape(coordinates, (2, Nx*Ny)) |> transpose
    
    return lattice, coordinates
end

square_lattice (generic function with 1 method)

In [12]:
function neighbors(Nx, Ny, periodicity)
    if periodicity == 0
        lattice = square_lattice(Nx,Ny)[1]
        Neighbors = []   
        for j in 0:Ny-1
            for i in 0:Nx-1
                x = [lattice[mod(j,Ny),mod(i-1,Nx)],lattice[mod(j+1,Ny),mod(i,Nx)],lattice[mod(j,Ny),mod(i+1,Nx)],lattice[mod(j-1,Ny),mod(i,Nx)]]
                x = unique(x)
                push!(Neighbors,x)
            end
        end
    else
        println("Hard-Core B.C. isn't ready not yet!")
    end
    return Neighbors
end

neighbors (generic function with 1 method)

In [13]:
# Hard-Core için yan yana push edemedim. Şimdlik kalsın
# Neig = []   
#     for j in 0:Ny-1
#         for i in 0:Nx-1
#             if j == 0 || i == 0 || j == Ny-1 || i == Nx-1 
#                 if j != 0
#                     push!(Neig, [lattice[j-1,i]])  
#                 end
#                 if i != 0
#                     push!(Neig, [lattice[j,i-1]])  
#                 end
#                 if j != Ny-1
#                     push!(Neig, [lattice[j+1,i]])  
#                 end
#                 if i != Nx-1
#                     push!(Neig, [lattice[j,i+1]])  
#                 end
#             else
#                 x = [
#                     lattice[j,i-1],
#                     lattice[j+1,i],
#                     lattice[j,i+1],
#                     lattice[j-1,i]
#                     ]
#                 push!(Neig,x)
#             end
#         end
#     end
# Neig

In [14]:
function Hofstadter_SP(Nx, Ny, alpha ,periodicity)
    
    neig = neighbors(Nx, Ny, 0)
    coordinates = square_lattice(Nx, Ny)[2]
    
    N = Nx*Ny
    t=-1
    H = zeros(Complex{Float64},Nx*Ny,Nx*Ny)
    
    for m in 1:N
        for n in 1:N
            if m in neig[n] 
                if abs(coordinates[m,1]-coordinates[n,1])==Nx-1
                    if coordinates[m,1] > coordinates[n,1]
                        H[m,n] = t*exp(-1im*2*pi*alpha*coordinates[m,2])
                    elseif coordinates[m,1] < coordinates[n,1]
                        H[m,n] = t*exp(1im*2*pi*alpha*coordinates[m,2])
                    end
                else
                    if coordinates[m,1] > coordinates[n,1]
                        H[m,n] = t*exp(1im*2*pi*alpha*coordinates[m,2])
                    elseif coordinates[m,1] < coordinates[n,1]
                        H[m,n] = t*exp(-1im*2*pi*alpha*coordinates[m,2])
                    else
                        H[m,n] = t*exp(0)
                    end
                end
            else
                H[m,n] = 0
            end
        end
    end
    
    return H
end

hofstadter_sP (generic function with 1 method)